In [ ]:
# Xin quyen Google drive
from google.colab import drive
drive.mount('/content/ggdrive')

Mounted at /content/ggdrive


In [ ]:
import pandas as pd

df = pd.read_csv(
    '/content/ggdrive/MyDrive/Phân tích dữ liệu lớn/Student Depression Dataset.csv'
)
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [ ]:
#Tóm lược dữ liệu
df_num = df.select_dtypes(include='number')
data_num = df_num.describe()
data_num.head(10)

,id,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27898.000000,27901.000000
mean,70442.149421,25.822300,3.141214,0.000430,7.656104,2.943837,0.000681,7.156984,3.139867,0.585499
std,40641.175216,4.905687,1.381465,0.043992,1.470707,1.361148,0.044394,3.707642,1.437347,0.492645
min,2.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,35039.000000,21.000000,2.000000,0.000000,6.290000,2.000000,0.000000,4.000000,2.000000,0.000000
50%,70684.000000,25.000000,3.000000,0.000000,7.770000,3.000000,0.000000,8.000000,3.000000,1.000000
75%,105818.000000,30.000000,4.000000,0.000000,8.920000,4.000000,0.000000,10.000000,4.000000,1.000000
max,140699.000000,59.000000,5.000000,5.000000,10.000000,5.000000,4.000000,12.000000,5.000000,1.000000


In [ ]:
#Đếm dữ liệu khuyết
df.isnull().sum()

,0
id,0
Gender,0
Age,0
City,0
Profession,0
Academic Pressure,0
Work Pressure,0
CGPA,0
Study Satisfaction,0
Job Satisfaction,0


In [ ]:
#Thay giá trị khuyết bằng giá trị trung bình
df['Financial Stress'] = df['Financial Stress'].fillna(df['Financial Stress'].mean())
#Đếm xem còn giá trị khuyết nữa hay không
df.isnull().sum()

,0
id,0
Gender,0
Age,0
City,0
Profession,0
Academic Pressure,0
Work Pressure,0
CGPA,0
Study Satisfaction,0
Job Satisfaction,0


In [ ]:
# Làm sạch và xử lý outliers
# Processing outliers
import numpy as np

z_threshold = 3
float_cols = df.select_dtypes(include=['number']).columns

for col in float_cols:
    # Tính trung bình và độ lệch chuẩn của cột
    mean = df[col].mean()
    std = df[col].std()

    # Tính Z-score
    z_scores = (df[col] - mean) / std

    # Xác định các chỉ số vượt ngưỡng Z-score
    index_lower = np.where(z_scores < -z_threshold)[0]
    index_upper = np.where(z_scores > z_threshold)[0]

    # In số lượng outliers
    print(col + ":", len(index_lower), len(index_upper))

    # Xử lý outliers: Thay thế bằng ngưỡng
    lower_threshold = mean - z_threshold * std
    upper_threshold = mean + z_threshold * std
    df.loc[index_lower, col] = lower_threshold
    df.loc[index_upper, col] = upper_threshold


id: 0 0
Age: 0 19
Academic Pressure: 0 0
Work Pressure: 0 3
CGPA: 9 0
Study Satisfaction: 0 0
Job Satisfaction: 0 8
Work/Study Hours: 0 0
Financial Stress: 0 0
Depression: 0 0


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# 1. Tách nhãn
df_class_0 = df_clean[df_clean['Depression'] == 0]
df_class_1 = df_clean[df_clean['Depression'] == 1]

# 2. Loại outliers trong nhãn 1 (ví dụ với cột CGPA)
Q1 = df_class_1['CGPA'].quantile(0.25)
Q3 = df_class_1['CGPA'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_class_1_filtered = df_class_1[
    (df_class_1['CGPA'] >= lower_bound) & (df_class_1['CGPA'] <= upper_bound)
]

# ✅ 3. Gộp lại nhãn 0 và nhãn 1 đã loại outliers
df_smote_input = pd.concat([df_class_0, df_class_1_filtered], ignore_index=True)

# 4. Tách X và y
X = df_smote_input.drop(columns=['Depression'])
y = df_smote_input['Depression']

# 5. Encode dữ liệu chuỗi
X_encoded = X.copy()
for col in X_encoded.select_dtypes(include='object').columns:
    X_encoded[col] = LabelEncoder().fit_transform(X_encoded[col])

# 6. SMOTE
smote = SMOTE(sampling_strategy=1.0, random_state=42)  # cân bằng 1:1
X_resampled, y_resampled = smote.fit_resample(X_encoded, y)

# 7. Tạo dataframe kết quả
df_resampled = pd.DataFrame(X_resampled, columns=X_encoded.columns)
df_resampled['Depression'] = y_resampled

# Xem kết quả
from collections import Counter
print("✅ Sau khi SMOTE:", Counter(y_resampled))
df_resampled.head(100)


✅ Sau khi SMOTE: Counter({0: 16332, 1: 16332})
           id  Gender        Age  City  Profession  Academic Pressure  \
0           8       0  24.000000     3          11           2.000000   
1          26       1  31.000000    44          11           3.000000   
2          32       0  25.000000    16          11           4.000000   
3          33       1  29.000000    39          11           2.000000   
4          52       1  30.000000    46          11           3.000000   
...       ...     ...        ...   ...         ...                ...   
32659  124465       1  31.125755    21          11           1.874245   
32660  118217       1  19.680801     3          11           4.336160   
32661  126010       1  28.925408    40          11           2.000000   
32662   73349       0  28.080184     6          11           2.020046   
32663   37729       0  28.825689    24          11           3.000000   

       Work Pressure      CGPA  Study Satisfaction  Job Satisfaction  \
0   

Dự báo


In [ ]:
# =========================
# 1. THƯ VIỆN & DỮ LIỆU
# =========================
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

# Đọc dữ liệu từ Google Drive
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv(
    '/content/drive/MyDrive/Phân tích dữ liệu lớn/Student Depression Dataset.csv'
)

# =========================
# 2. TIỀN XỬ LÝ DỮ LIỆU
# =========================

features = [
    "Age", "Gender", "CGPA", "Academic Pressure", "Study Satisfaction",
    "Sleep Duration", "Financial Stress", "Family History of Mental Illness"
]
target = "Depression"

X = df[features].copy()
y = df[target]

# Nhóm định lượng
numerical_cols = ["Age", "CGPA", "Academic Pressure", "Study Satisfaction", "Financial Stress"]

# Chuyển nhị phân
X["Gender"] = X["Gender"].map({"Male": 1, "Female": 0})
X["Family History of Mental Illness"] = X["Family History of Mental Illness"].map({"Yes": 1, "No": 0})

# Ánh xạ thứ tự
sleep_map = {
    "Less than 5 hours": 0,
    "5-6 hours": 1,
    "7-8 hours": 2,
    "More than 8 hours": 3
}
X["Sleep Duration"] = X["Sleep Duration"].map(sleep_map)

# ⚠️ Xử lý NaN sau khi mapping
X.dropna(inplace=True)
y = y.loc[X.index]

# =========================
# 3. CHIA DỮ LIỆU & CHUẨN HÓA
# =========================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Tránh cảnh báo khi gán
X_train = X_train.copy()
X_test = X_test.copy()

# Chuẩn hóa dữ liệu số
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Chuyển về float để dùng với SMOTE
X_train = X_train.astype(float)

# =========================
# 4. ÁP DỤNG SMOTE
# =========================

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# =========================
# 5. HUẤN LUYỆN & LƯU MÔ HÌNH
# =========================

model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

joblib.dump(model, "logistic_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Mô hình đã được huấn luyện và lưu thành công!")

# =========================
# 6. DỰ BÁO TỪ DỮ LIỆU NGƯỜI DÙNG
# =========================

print("\n=== DỰ BÁO TRẦM CẢM ===")
age = int(input("Tuổi: "))
gender = input("Giới tính (Male/Female): ")
cgpa = float(input("Điểm CGPA: "))
academic_pressure = int(input("Áp lực học tập (1-5): "))
study_satisfaction = int(input("Hài lòng việc học (1-5): "))
sleep_duration = input("Thời gian ngủ (Less than 5 hours / 5-6 hours / 7-8 hours / More than 8 hours): ")
financial_stress = int(input("Áp lực tài chính (1-5): "))
family_history = input("Gia đình có tiền sử bệnh tâm thần? (Yes/No): ")

# Tiền xử lý đầu vào người dùng
sleep_map = {
    "Less than 5 hours": 0,
    "5-6 hours": 1,
    "7-8 hours": 2,
    "More than 8 hours": 3
}
gender_map = {"Male": 1, "Female": 0}
history_map = {"Yes": 1, "No": 0}

input_df = pd.DataFrame([{
    "Age": age,
    "Gender": gender_map.get(gender, 0),
    "CGPA": cgpa,
    "Academic Pressure": academic_pressure,
    "Study Satisfaction": study_satisfaction,
    "Sleep Duration": sleep_map.get(sleep_duration, 1),
    "Financial Stress": financial_stress,
    "Family History of Mental Illness": history_map.get(family_history, 0)
}])

# Chuẩn hóa dữ liệu đầu vào
scaler = joblib.load("scaler.pkl")
input_df[numerical_cols] = scaler.transform(input_df[numerical_cols])

# Dự đoán với mô hình đã huấn luyện
model = joblib.load("logistic_model.pkl")
prediction = model.predict(input_df)[0]
probability = model.predict_proba(input_df)[0][1]

# Kết quả
print("\n=== KẾT QUẢ DỰ BÁO ===")
print("Tình trạng trầm cảm:", "Có trầm cảm" if prediction == 1 else "Không trầm cảm")
print(f"Xác suất mắc trầm cảm: {probability*100:.2f}%")

Mounted at /content/drive
Mô hình đã được huấn luyện và lưu thành công!

=== DỰ BÁO TRẦM CẢM ===
Tuổi: 12
Giới tính (Male/Female): Male
Điểm CGPA: 4
Áp lực học tập (1-5): 1
Hài lòng việc học (1-5): 1
Thời gian ngủ (Less than 5 hours / 5-6 hours / 7-8 hours / More than 8 hours): 5-6 hours
Áp lực tài chính (1-5): 1
Gia đình có tiền sử bệnh tâm thần? (Yes/No): No

=== KẾT QUẢ DỰ BÁO ===
Tình trạng trầm cảm: Không trầm cảm
Xác suất mắc trầm cảm: 25.03%
